# Import Dependenciesg

In [1]:
import os

import tensorflow as tf
import pandas as pd
import keras

2023-10-18 22:55:18.806975: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-18 22:55:18.997927: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-18 22:55:18.999001: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 22:55:20.535091: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
keras.__version__

'2.13.1'

In [3]:
tf.__version__

'2.13.0'

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


# Read in the Data 

In [6]:
rawData = pd.read_excel("Full-Data-Set-UnderOver-2020-21.xlsx")
rawData = rawData.sample(frac=1)

In [7]:
rawData.describe()

,Unnamed: 0,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,...,BLK_RANK.1,BLKA_RANK.1,PF_RANK.1,PFD_RANK.1,PTS_RANK.1,PLUS_MINUS_RANK.1,Score,Home-Team-Win,OU,OU-Cover
count,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,...,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000
mean,8178.500000,42.738538,22.225883,20.512654,0.513536,48.353754,37.952763,83.493654,0.454603,8.192689,...,15.307984,15.168480,15.353344,15.477809,15.275095,15.036007,204.782186,0.591148,204.598392,0.515344
std,4722.292187,24.007672,15.430625,13.403097,0.183969,0.325391,2.318313,3.831431,0.019487,2.528331,...,8.621444,8.633495,8.640666,8.612438,8.633568,8.664541,22.006642,0.491637,21.729718,0.526352
min,0.000000,1.000000,0.000000,0.000000,0.000000,48.000000,28.000000,67.300000,0.339000,0.700000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,22.000000,0.000000,1.000000,0.000000
25%,4089.250000,22.000000,10.000000,9.000000,0.382000,48.100000,36.400000,80.900000,0.442000,6.300000,...,8.000000,8.000000,8.000000,8.000000,8.000000,7.000000,190.000000,0.000000,194.500000,0.000000
50%,8178.500000,43.000000,19.000000,19.000000,0.520000,48.300000,37.800000,83.400000,0.454000,7.900000,...,15.000000,15.000000,15.000000,16.000000,15.000000,15.000000,204.000000,1.000000,203.500000,1.000000
75%,12267.750000,64.000000,32.000000,29.000000,0.645000,48.500000,39.300000,86.100000,0.467000,9.900000,...,23.000000,23.000000,23.000000,23.000000,23.000000,22.000000,219.000000,1.000000,214.000000,1.000000
max,16357.000000,82.000000,72.000000,69.000000,1.000000,55.500000,50.000000,100.000000,0.571000,20.000000,...,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,329.000000,1.000000,1955.500000,2.000000


## Clean the Data

In [8]:
filteredData = rawData.drop(["Unnamed: 0"], axis = 1)
filteredData = filteredData.drop(["Score"], axis = 1)
filteredData = filteredData.drop(["GP"], axis = 1)
filteredData = filteredData.drop(["GP.1"], axis = 1)
filteredData = filteredData.drop(["Home-Team-Win"], axis = 1)
filteredData = filteredData.drop(["TEAM_NAME"], axis = 1)
filteredData = filteredData.drop(["Date"], axis = 1)
filteredData = filteredData.drop(["MIN", "W", "W.1", "L", "L.1", "PLUS_MINUS", "PLUS_MINUS.1", "PLUS_MINUS_RANK", "PLUS_MINUS_RANK.1", "W_RANK", "W_RANK.1", "L_RANK", "L_RANK.1"], axis = 1)
filteredData = filteredData.drop(["MIN.1"], axis = 1)
filteredData = filteredData.drop(["MIN_RANK.1"], axis = 1)
filteredData = filteredData.drop(["MIN_RANK"], axis = 1)
filteredData = filteredData.drop(["TEAM_NAME.1"], axis = 1)
filteredData = filteredData.drop(["Date.1"], axis = 1)
filteredData = filteredData.drop(["OU"], axis = 1)
filteredData = filteredData.drop(["OU-Cover"], axis = 1)
filteredData = filteredData.drop(["GP_RANK"], axis = 1)
filteredData = filteredData.drop(["GP_RANK.1"], axis = 1)

# Train the Model

#### Get the outputs

In [9]:
output = rawData['Home-Team-Win']

In [10]:
#get 80% for training 20% for testing
print(len(filteredData))
test_len = int(len(filteredData) * 0.8)
x_train = filteredData.take(range(test_len))
x_test = filteredData.take(range(test_len, len(filteredData)))
y_train = output.take(range(test_len))
y_test = output.take(range(test_len, len(filteredData)))
print(y_train.shape)
print(y_test[0].shape)
print(y_test.shape)

16358
(13086,)


KeyError: 0

In [ ]:
test_len = int(len(filteredData) * 0.9)
x_train = filteredData.take(range(test_len))
x_test = filteredData.take(range(test_len, len(filteredData)))
y_train = output.take(range(test_len))
y_test = output.take(range(test_len, len(filteredData)))

In [ ]:
import datetime
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(170, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dropout(rate=.25))
model.add(tf.keras.layers.Dense(150, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dense(75, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dropout(rate=.15))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dense(15, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax, name="output_layer"))


In [ ]:
model.build( x_train.shape )
model.summary()

In [ ]:
from keras.optimizers import Adam
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.compile(Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, epochs=60, validation_split=0.30, batch_size=3_000, verbose=2, shuffle=True, use_multiprocessing=True, callbacks=[tensorboard_callback])

NameError: name 'model' is not defined

In [ ]:
todayPredictions = model.predict([x_test])

In [ ]:
model = tf.keras.models.load_model("./trained_models/v2")

In [ ]:
import os
import numpy as np
model_dir = "../API/src/data"
for file in os.listdir(model_dir):
    if not file.endswith(".csv") or not file.__contains__("2022"):
        continue

    teamStats = pd.read_csv(f"{model_dir}/{file}")
    filteredTeamStats = teamStats.drop(["TEAM_NAME", "TEAM_ID","GP","GP_RANK","CFID", "MIN", "CFPARAMS", "MIN_RANK", "MIN_RANK.1"], axis = 1)
    filteredTeamStats = filteredTeamStats.drop(["W","L","PLUS_MINUS","W_RANK", "L_RANK", "PLUS_MINUS_RANK",], axis = 1)
    filteredTeamStats = filteredTeamStats.drop(["TEAM_NAME.1", "TEAM_ID.1","GP.1","GP_RANK.1","CFID.1", "MIN.1", "CFPARAMS.1",], axis = 1)
    filteredTeamStats = filteredTeamStats.drop(["W.1","L.1","PLUS_MINUS.1","W_RANK.1", "L_RANK.1", "PLUS_MINUS_RANK.1",], axis = 1)
    todayPredictions = model.predict(filteredTeamStats)

    print(f"{file}")
    for i in range (0,len(todayPredictions)):
        away_team_perc = todayPredictions[i][0]
        home_team_perc = todayPredictions[i][1]
        print(f"{teamStats['TEAM_NAME'][i]} {home_team_perc} vs {teamStats['TEAM_NAME.1'][i]} {away_team_perc}")
        if np.argmax(todayPredictions[i]) == 1:
            print(f"{teamStats['TEAM_NAME'][i]} win")
        else:
            print(f"{teamStats['TEAM_NAME.1'][i]} win ")
        print()
        print()


# Save the Model

In [ ]:
import datetime
model.save(f"model_{datetime.datetime.utcnow()}")